# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

Install Helm

In [ ]:
!helm init

Label the node to allow load testing to run on it

In [ ]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

MINIKUBE_IP=commands.getoutput('minikube ip')

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [ ]:
!helm install seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --repo https://storage.googleapis.com/seldon-charts

Check all services are running before proceeding.

In [ ]:
!kubectl get pods

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [ ]:
!kubectl create -f resources/model_invalid1.json

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [ ]:
!kubectl create -f resources/model_invalid2.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!kubectl delete -f resources/model_invalid2.json

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [ ]:
!cat resources/model.json

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [ ]:
!kubectl apply -f resources/model.json

In [ ]:
!kubectl get seldondeployments

In [ ]:
!kubectl describe seldondeployments seldon-deployment-example 

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

## Get predictions

#### REST Request

In [ ]:
rest_request()

#### gRPC Request

In [ ]:
grpc_request()

## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [ ]:
!cat resources/model_with_canary.json

In [ ]:
!kubectl apply -f resources/model_with_canary.json

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

#### REST Request

In [ ]:
rest_request()

#### gRPC request

In [ ]:
grpc_request()

## Load test

Start a load test which will post REST requests at 10 requests per second.

!helm install seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --repo https://storage.googleapis.com/seldon-charts

You can view an analytics dashboard inside the cluster at http://192.168.99.100:30034/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json

In [ ]:
!helm delete seldon-core --purge